In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.DCVC_net import DCVC_net
import torch
from torchvision import transforms
import numpy as np
import pathlib
import os
import matplotlib.pyplot as plt
import wandb
import tqdm
from torchnet import meter

/home/jatin/miniconda3/envs/DCVC/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BATCH_SIZE = 4
DATA_DIR = pathlib.Path('../uvg/Beauty_PNG_1024/')
DEVICE = torch.device('cuda')
DEVICE

device(type='cuda')

In [4]:
video_net = DCVC_net()

In [5]:
# # load the good weights
# video_net.opticFlow = torch.load('../DCVC/optflow.pth')
# video_net.mvEncoder = torch.load('../DCVC/mvenc.pth')
# video_net.mvDecoder_part1 = torch.load('../DCVC/mvDecoder_part1.pth')
# video_net.mvDecoder_part2 = torch.load('../DCVC/mvDecoder_part2.pth')
# video_net.feature_extract = torch.load('../DCVC/feature_extract.pth')
# video_net.context_refine = torch.load('../DCVC/context_refine.pth')
# # video_net.contextualDecoder_part1 = torch.load('../DCVC/contextualDecoder_part1.pth')

video_net.load_state_dict(torch.load('checkpoints/model_dcvc_quality_3_psnr.pth'))
# print('Reminder to load optimizer too')
# # optimizer = torch.optim.Adam(video_net.parameters(), lr=wandb.config.learning_rate)

# video_net.opticFlow.requires_grad_ = False
# video_net.mvEncoder.requires_grad_ = False
# video_net.mvDecoder_part1.requires_grad_ = False
# video_net.mvDecoder_part2.requires_grad_ = False
# video_net.feature_extract.requires_grad_ = False
# video_net.context_refine.requires_grad_ = False
# # video_net.contextualDecoder_part1.requires_grad_ = False


<All keys matched successfully>

In [6]:
class UVGDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, crop_size=None, deterministic=False):
        self.data_dir = data_dir
        self.crop_size = crop_size
        self.deterministic = deterministic
        self.numframes = len(os.listdir(data_dir))
        if crop_size is not None:
            self.transforms = torch.nn.Sequential(
                transforms.CenterCrop(crop_size)
            )
        else:
            self.transforms = None
       
    def __getitem__(self, i):
        interval = 1
        # this is 1-indexed on disk
        i += 1
        int_as_str = f'{i}'.zfill(5)
        ref = plt.imread(self.data_dir / f'im{int_as_str}.png')
        int_as_str = f'{i + interval}'.zfill(5)
        im = plt.imread(self.data_dir / f'im{int_as_str}.png')
        imgs = [ref, im]

        # plt.imread should make inputs in [0, 1] for us
        imgs = np.stack(imgs, axis=0)
        # bring RGB channels in front
        imgs = imgs.transpose(0, 3, 1, 2)
        imgs = torch.FloatTensor(imgs)
        if self.transforms:
            imgs = self.transforms(imgs)
        return imgs

    def __len__(self):
        return self.numframes - 1

ds = UVGDataset(DATA_DIR, crop_size=704)
dl = torch.utils.data.DataLoader(
    ds,
    shuffle=True,
    batch_size=BATCH_SIZE,
    num_workers=6,
    prefetch_factor=5
)

In [7]:
video_net = video_net.to(DEVICE)

In [8]:
def count_parameters(model):
    """Return number of parameters in a model"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(video_net)} trainable parameters')

The model has 7944448 trainable parameters


In [9]:
def test_epoch(model, dl, compress):
    model.eval()
    mse_criterion = torch.nn.MSELoss()
    
    mse_meter = meter.AverageValueMeter()
    bpp_meter = meter.AverageValueMeter()
    bpp_mv_y_meter = meter.AverageValueMeter()
    bpp_mv_z_meter = meter.AverageValueMeter()
    bpp_y_meter = meter.AverageValueMeter()
    bpp_z_meter = meter.AverageValueMeter()
    
    pbar = tqdm.tqdm(dl)
    for i, x in enumerate(pbar):
        x = x.to(DEVICE)
        ref = x[:,0]
        im = x[:,1]
        with torch.no_grad():
            preds_dict = model(ref, im, compress=compress)
        preds = preds_dict['recon_image']

        mse_ls = mse_criterion(preds, im)
        mse_meter.add(mse_ls.item())
        bpp_meter.add(preds_dict['bpp'].item())
        bpp_mv_y_meter.add(preds_dict['bpp_mv_y'].item())
        bpp_mv_z_meter.add(preds_dict['bpp_mv_z'].item())
        bpp_y_meter.add(preds_dict['bpp_y'].item())
        bpp_z_meter.add(preds_dict['bpp_z'].item())
        if i % 1 == 0:
            avg_psnr = round(-10.0*np.log10(mse_meter.value()[0]), 6)
            avg_bpp = round(bpp_meter.value()[0], 6)
            avg_bpp_mv_y = round(bpp_mv_y_meter.value()[0], 6)
            avg_bpp_mv_z = round(bpp_mv_z_meter.value()[0], 6)
            avg_bpp_y = round(bpp_y_meter.value()[0], 6)
            avg_bpp_z = round(bpp_z_meter.value()[0], 6)
            msg = (
                f'Avg PSNR: {avg_psnr}, bpp: {avg_bpp}, bpp_mv_y: {avg_bpp_mv_y}, avg_bpp_mv_z: {avg_bpp_mv_z} '
                f'avg_bpp_y: {avg_bpp_y}, avg_bpp_z: {avg_bpp_z}'
            )
            pbar.set_description(msg)
            
    return avg_loss
            

In [10]:
test_epoch(video_net, dl, compress=True)

Avg PSNR: 35.653138, bpp: 0.068851, bpp_mv_y: 0.026817, avg_bpp_mv_z: 0.000759 avg_bpp_y: 0.040065, avg_bpp_z: 0.001209:  12%|█▏        | 18/150 [00:14<01:48,  1.21it/s]
Traceback (most recent call last):
  File "/home/jatin/miniconda3/envs/DCVC/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/jatin/miniconda3/envs/DCVC/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/jatin/miniconda3/envs/DCVC/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/home/jatin/miniconda3/envs/DCVC/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [24]:
test_epoch(video_net, dl, compress=True)

Avg PSNR: 35.690355, bpp: 0.07611, bpp_mv_y: 0.028113, avg_bpp_mv_z: 0.000759 avg_bpp_y: 0.046028, avg_bpp_z: 0.001211:  17%|█▋        | 25/150 [00:19<01:38,  1.27it/s] 


KeyboardInterrupt: 

In [25]:
test_epoch(video_net, dl, compress=False)

Avg PSNR: 35.724303, bpp: 0.073545, bpp_mv_y: 0.026693, avg_bpp_mv_z: 0.000759 avg_bpp_y: 0.044851, avg_bpp_z: 0.001242:   8%|▊         | 12/150 [00:10<02:02,  1.13it/s]


KeyboardInterrupt: 

In [11]:
test_epoch(video_net, dl, criterion, compress=True)

Avg PSNR/Loss, Batch Loss: (35.778809, 0.000264, 0.000276):  12%|█▏        | 18/150 [00:15<01:52,  1.17it/s]
Traceback (most recent call last):
  File "/home/jatin/miniconda3/envs/DCVC/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/jatin/miniconda3/envs/DCVC/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/jatin/miniconda3/envs/DCVC/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/home/jatin/miniconda3/envs/DCVC/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/jatin/miniconda3/envs/DCVC/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/jatin/miniconda3/envs/DCVC/lib/python3.8/multiprocessing/connection.py", line 200, in 

KeyboardInterrupt: 

In [11]:
test_epoch(video_net, dl, criterion, compress=True)

Avg PSNR/Loss, Batch Loss: (35.664951, 0.000271, 0.000279):  11%|█▏        | 17/150 [00:15<01:58,  1.12it/s]


KeyboardInterrupt: 

In [11]:
test_epoch(video_net, dl, criterion, compress=False)

Avg PSNR/Loss, Batch Loss: (35.729129, 0.000267, 0.000264):  15%|█▍        | 22/150 [00:18<01:47,  1.19it/s]


KeyboardInterrupt: 

In [14]:
for i in range(1, wandb.config.epochs):
    avg_loss, had_err, err_x = train_epoch(video_net, i, dl, optimizer, criterion)
    if had_err:
        print('Breaking out of train loop for debugging...')
        break
    torch.save(
        {'model': video_net.state_dict(), 'optimizer': optimizer.state_dict()},
        SAVE_FOLDER / f"dcvc_epoch={i}_avg_loss={avg_loss}.pt",
    )
    

Avg PSNR/Loss, Batch Loss: (36.6044, 0.0002, 0.0003):  17%|█▋        | 4000/22926 [27:58<2:25:19,  2.17it/s]

Saving model with avg loss 0.0002


Avg PSNR/Loss, Batch Loss: (37.2588, 0.0002, 0.0002):  30%|██▉       | 6799/22926 [47:33<1:52:49,  2.38it/s]wandb: Network error (ReadTimeout), entering retry loop.
Avg PSNR/Loss, Batch Loss: (37.4266, 0.0002, 0.0002):  35%|███▍      | 8000/22926 [55:56<1:53:00,  2.20it/s]

Saving model with avg loss 0.0002


Avg PSNR/Loss, Batch Loss: (37.6845, 0.0002, 0.0001):  52%|█████▏    | 12000/22926 [1:23:54<1:22:29,  2.21it/s]

...

Avg PSNR/Loss, Batch Loss: (37.9732, 0.0002, 0.0001):  72%|███████▏  | 16527/22926 [1:55:34<44:40,  2.39it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Avg PSNR/Loss, Batch Loss: (38.4354, 0.0001, 0.0001):  10%|█         | 2398/22926 [16:47<2:23:19,  2.39it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Avg PSNR/Loss, Batch Loss: (38.893, 0.0001, 0.0001):  47%|████▋     | 10822/22926 [1:15:41<1:24:40,  2.38it/s] IOPub message rate exc

Saving model with avg loss 0.0001


Avg PSNR/Loss, Batch Loss: (40.0046, 0.0001, 0.0):  23%|██▎       | 5243/22926 [36:40<2:03:49,  2.38it/s]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Avg PSNR/Loss, Batch Loss: (40.6615, 0.0001, 0.0):  52%|█████▏    | 12000/22926 [1:23:55<1:26:38,  2.10it/s]   

Saving model with avg loss 0.0001


Avg PSNR/Loss, Batch Loss: (40.7101, 0.0001, 0.0):  58%|█████▊    | 13315/22926 [1:33:07<1:07:13,  2.38it/s]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Avg PSNR/Loss, Batch Loss: (41.0836, 0.0001, 0.0):  93%|█████████▎| 21374/22926 [2:29:29<10:51,  2.38it/s]   wandb: Network error (ReadTimeout), entering retry loop.
Avg PSNR/Loss, Batch Loss: (40.952, 0.0001, 0.0):  95%|█████████▌| 21872/22926 [2:32:58<07:21,  2.39it/s]    IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
Notebo

In [15]:
wandb.finish()

train/avg_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▇▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/avg_psnr,████████████████▁▁▁▁▁▇▇▇▇▇▇▇██▆▆▆▆▆▆▆▆▆▆
train/batch_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▅▂▃▂▂▂▂▁▁▂▁▁▁█▂▂▁▁▂▁▁▂▁
train/epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███████████████████
train/avg_loss,0.0047
train/avg_psnr,23.2678
train/batch_loss,0.0005
train/epoch,2
